In [9]:
import pandas as pd
import ast
import re

In [10]:
with open('../raw/PEP_LIFE.data', 'r') as f:
    contenido = f.read()

In [11]:
data = ast.literal_eval(contenido)

In [12]:
rows = []
for key, value in data.items():
    sequence, modifications = key
    half_life_data = value['half_life_data']
    references = value['references']
    url_sources = value['url_sources']
    
    # Extraer datos de cada entrada en half_life_data
    for entry, url in half_life_data:
        row = {
            'Sequence': sequence,
            'Modifications': modifications,
            'Test Sample': entry['test_sample'],
            'Vivo/Vitro': entry['vivo_vitro'],
            'Half Life (seconds)': entry['half_life_seconds'],
            'Reference URL': references[0] if references else None,
            'Source URL': url,
            'Other Sources': ', '.join(url_sources)
        }
        rows.append(row)


In [13]:
df = pd.DataFrame(data)

In [14]:
df_t = df.transpose()
df_t.reset_index(inplace=True)
df_t=df_t.filter(['level_0','half_life_data'])

In [15]:
df_t['half_life_data'] = df_t['half_life_data'].astype(str).str.replace(r'[\[\]():{}]', '', regex=True)
def extract_data(text):
    return re.findall(r"'(.*?)'", text)

extracted_data = df_t['half_life_data'].apply(extract_data)
df_expanded = pd.DataFrame(extracted_data.tolist())
df_expanded[1]=pd.to_numeric(df_expanded[1], errors='coerce')
df_t['half_life_data'] = df_expanded[1]

In [16]:
df_t = df_t.rename(columns={'level_0': 'sequence', 'half_life_data': 'half_life_seconds'})
df_t.to_csv('../datasets/PEP_LIFE.csv', index=False)